imports

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import numpy as np


dataset


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

169017344/169001437 [==============================] - 3s 0us/step


In [ ]:

def prepare_mnist_data(x , y):

  mnist = tf.data.Dataset.from_tensor_slices((x, y))

  mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))

  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  mnist = mnist.map(lambda img, target: ((img/128.)-1., target))

  #create one-hot targets
  mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=10)))

  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  mnist = mnist.cache()


  #shuffle, batch, prefetch
  mnist = mnist.shuffle(1000)
  mnist = mnist.batch(64)
  mnist = mnist.prefetch(20)

  #return preprocessed dataset
  return mnist

In [ ]:
train_dataset = prepare_mnist_data(x_train,y_train)
test_dataset = prepare_mnist_data(x_test,y_test)

print(train_dataset)
print(test_dataset)

<PrefetchDataset shapes: ((None, 32, 32, 3), (None, 1, 10)), types: (tf.float32, tf.float32)>
<PrefetchDataset shapes: ((None, 32, 32, 3), (None, 1, 10)), types: (tf.float32, tf.float32)>


Resnet

In [ ]:


from tensorflow.keras.layers import Dense,BatchNormalization

class My_resblock(tf.keras.Model):
    
    def __init__(self, my_kernal, my_filter ):
        super(My_resblock, self).__init__()

        # target 

        # conv 1 (3x3) ()32x32x3 ===> (32,32,32)
        self.convlayer_1 = tf.keras.layers.Conv2D(filters=my_filter[0], kernel_size=my_kernal[0], strides=(1, 1), padding="same",activation=tf.nn.relu, 
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        # batch (32,32,32)
        self.batchnorm_1 = BatchNormalization()


        #conv 2
        self.convlayer_2 = tf.keras.layers.Conv2D(filters=my_filter[1], kernel_size=my_kernal[1], strides=(1, 1), padding="same",activation=tf.nn.softmax,
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        # batch(32,32,32)
        self.batchnorm_2 = BatchNormalization()   

        
        



       

    @tf.function
    def call(self, inputs):
        

        x = self.convlayer_1(inputs)
        x = self.batchnorm_1(x)
        x = self.convlayer_2(x)
        x = self.batchnorm_2(x)

        x += inputs
         # we have to add it up(32,32,32) + (32,32,3)       

        return x
    

model

In [ ]:
from tensorflow.keras.layers import Dense,BatchNormalization

class My_resnet(tf.keras.Model):
    
    def __init__(self):
        super(My_resnet, self).__init__()
        

        self.convlayer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding="same",activation=tf.nn.relu, 
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        self.My_resblock_1 = My_resblock(my_kernal= (1,3), my_filter = (16,32))
       

    @tf.function
    def call(self, inputs):
        x = self.convlayer_1(inputs)
        x = self.My_resblock_1(x)
        return x
    

In [ ]:
dope = tf.ones((1,5,5,256))

In [ ]:
my_net = My_resnet()

my_net(dope).shape

TensorShape([1, 5, 5, 32])

Densenet


In [ ]:
from tensorflow.keras.layers import Dense,BatchNormalization

class My_denseblock(tf.keras.Model):
    
    def __init__(self, my_kernal=(1,3), my_filter=(16,32) ):
        super(My_denseblock, self).__init__()

        # target 
        self.batchnorm_1 = BatchNormalization()

        # conv 1 (3x3) ()32x32x3 ===> (32,32,32)
        self.convlayer_1 = tf.keras.layers.Conv2D(filters=my_filter[0], kernel_size=my_kernal[0], strides=(1, 1), padding="same",activation=tf.nn.relu, 
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        # batch (32,32,32)

        self.batchnorm_2 = BatchNormalization()
        


        #conv 2
        self.convlayer_2 = tf.keras.layers.Conv2D(filters=my_filter[1], kernel_size=my_kernal[1], strides=(1, 1), padding="same",activation=tf.nn.softmax,
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        # batch(32,32,32)
        self.batchnorm_3 = BatchNormalization()   

        
        



       

    @tf.function
    def call(self, inputs):
        

        x = self.batchnorm_1(inputs)

        x = self.convlayer_1(x)
        x = self.batchnorm_2(x)
        x = self.convlayer_2(x)
        x = self.batchnorm_3(x)

        x =tf.keras.layers.Concatenate(axis=3)([x, inputs])
         # we have to add it up(32,32,32) + (32,32,3)       

        return x



In [ ]:
from tensorflow.keras.layers import Dense,BatchNormalization

class My_densenet(tf.keras.Model):
    
    def __init__(self, num_blocks):
        super(My_densenet, self).__init__()
        

        self.convlayer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding="same",activation=tf.nn.relu, 
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        
        self.blocks = [(My_denseblock(),My_transition()) for _ in range(num_blocks)]
        #self.My_densenet_1 = My_denseblock(my_kernal= (1,3), my_filter = (16,32))
        self.my_global = tf.keras.layers.GlobalAveragePooling2D()

        self.out_put = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

        
    @tf.function
    def call(self, inputs):
        x = self.convlayer_1(inputs)
        for dense_block, transition in self.blocks:
          x = dense_block(x)
          x = transition(x)

        x = self.my_global(x)
        x = self.out_put(x)

        #x = self.blocks(x)
        return x

Transition

In [ ]:
from tensorflow.keras.layers import Dense,BatchNormalization

class My_transition(tf.keras.Model):
    
    def __init__(self, filter_number= 32):
        super(My_transition, self).__init__()
        

        self.convlayer_1 = tf.keras.layers.Conv2D(filters=filter_number, kernel_size=(1,1), strides=(1, 1), padding="valid",activation=tf.nn.relu, 
                                                  kernel_regularizer = tf.keras.regularizers.l2(l2=0.01))
        
        self.poolinglayer_1 = tf.keras.layers.AvgPool2D(pool_size=(2,2) ,strides=(2, 2), padding="valid")
        
       

    @tf.function
    def call(self, inputs):
        x = self.convlayer_1(inputs)
        x = self.poolinglayer_1(x)
        return x

In [ ]:
dense = My_densenet(4)

In [ ]:
x = tf.ones((1,5,5,265))
dense(x)

ValueError: ignored

implementation of tenas and dense blocks


In [ ]:
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    #print(prediction)

    # model.losses then need to reduce to single value
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss



In [ ]:
def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input)
    #print(prediction.shape)
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [ ]:
tf.keras.backend.clear_session()

#For showcasing we only use a subset of the training and test data (generally use all of the available data!)
#train_dataset = train_dataset.take(1000)
#test_dataset = test_dataset.take(100)

### Hyperparameters
num_epochs = 10
learning_rate = 0.001

# Initialize the model.
model = My_Model()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: SGD with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_dataset:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)